In [42]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [43]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME' 'STATUS', 'SPECIAL_CONSIDERATIONS','ASK_AMT'.
clean_application_df = application_df.drop(columns = ['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS','ASK_AMT'])
clean_application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,1
...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,0
34296,T3,CompanySponsored,C2000,Preservation,Association,0,0
34297,T5,Independent,C3000,ProductDev,Association,0,1


In [44]:
# Determine the number of unique values in each column.
clean_application_df.nunique()

APPLICATION_TYPE    17
AFFILIATION          6
CLASSIFICATION      71
USE_CASE             5
ORGANIZATION         4
INCOME_AMT           9
IS_SUCCESSFUL        2
dtype: int64

In [45]:
type_count = clean_application_df['APPLICATION_TYPE'].value_counts()
type_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [46]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(type_count.iloc[8:].index)
cutoff_value = 528

# Replace in dataframe
for app in application_types_to_replace:
    clean_application_df['APPLICATION_TYPE'] = clean_application_df['APPLICATION_TYPE'].replace(app,"Other_app_type")

# Check to make sure binning was successful
clean_application_df['APPLICATION_TYPE'].value_counts()

T3                27037
T4                 1542
T6                 1216
T5                 1173
T19                1065
T8                  737
T7                  725
T10                 528
Other_app_type      276
Name: APPLICATION_TYPE, dtype: int64

In [47]:
# Look at CLASSIFICATION value counts for binning
classification_types = clean_application_df['CLASSIFICATION'].value_counts()
classification_types

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [48]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classifications_cleaned = classification_types.loc[classification_types > 1]
classifications_cleaned

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [49]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_types.iloc[5:].index)
cutoff_value_classifications = 1883

# Replace in dataframe
for cls in classifications_to_replace:
    clean_application_df['CLASSIFICATION'] = clean_application_df['CLASSIFICATION'].replace(cls,"Other_classification")

# Check to make sure binning was successful
clean_application_df['CLASSIFICATION'].value_counts()

C1000                   17326
C2000                    6074
C1200                    4837
Other_classification     2261
C3000                    1918
C2100                    1883
Name: CLASSIFICATION, dtype: int64

In [50]:
#convert the categorical data in numeric using get_dummies()
dummies = pd.get_dummies(clean_application_df)
dummies.head()

,IS_SUCCESSFUL,APPLICATION_TYPE_Other_app_type,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,0,0,0,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,1,0,0,0,1,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [51]:
# Split our preprocessed data into our features and target arrays
y = dummies['IS_SUCCESSFUL'].values
X = dummies.drop(columns='IS_SUCCESSFUL').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [52]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [53]:
# FIRST ATTEMPT
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features_total = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim = input_features_total, activation = "relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 8)                 320       
                                                                 
 dense_7 (Dense)             (None, 5)                 45        
                                                                 
 dense_8 (Dense)             (None, 1)                 6         
                                                                 
Total params: 371 (1.45 KB)
Trainable params: 371 (1.45 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [54]:
# Compile the model (FIRST ATTEMPT)
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [55]:
# Train the model (FIRST ATTEMPT)
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)


Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.6271 - accuracy: 0.6790
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5772 - accuracy: 0.7176
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5657 - accuracy: 0.7231
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5604 - accuracy: 0.7236
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5570 - accuracy: 0.7245
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5552 - accuracy: 0.7252
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5544 - accuracy: 0.7247
Epoch 8/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5536 - accuracy: 0.7254
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5528 - accuracy: 0.7258
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5523 - accura

In [56]:
# Evaluate the model using the test data (FIRST ATTEMPT)
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5554 - accuracy: 0.7289 - 530ms/epoch - 2ms/step
Loss: 0.5554225444793701, Accuracy: 0.728863000869751


In [57]:
# SECOND ATTEMPT
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features_total = len(X_train[0])
hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim = input_features_total, activation = "relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 10)                400       
                                                                 
 dense_10 (Dense)            (None, 10)                110       
                                                                 
 dense_11 (Dense)            (None, 1)                 11        
                                                                 
Total params: 521 (2.04 KB)
Trainable params: 521 (2.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [58]:
# Compile the model (SECOND ATTEMPT)
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [59]:
# Train the model (SECOND ATTEMPT)
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)


Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5949 - accuracy: 0.7029
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5614 - accuracy: 0.7284
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5571 - accuracy: 0.7287
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5548 - accuracy: 0.7286
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5534 - accuracy: 0.7303
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5524 - accuracy: 0.7302
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5519 - accuracy: 0.7304
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5509 - accuracy: 0.7320
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5505 - accuracy: 0.7309
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5501 - accura

In [60]:
# Evaluate the model using the test data (SECOND ATTEMPT)
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5540 - accuracy: 0.7280 - 519ms/epoch - 2ms/step
Loss: 0.5539721846580505, Accuracy: 0.7280466556549072


In [66]:
# THIRD ATTEMPT
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features_total = len(X_train[0])
hidden_nodes_layer1 = 25
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim = input_features_total, activation = "tanh"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation = "sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 25)                1000      
                                                                 
 dense_16 (Dense)            (None, 30)                780       
                                                                 
 dense_17 (Dense)            (None, 1)                 31        
                                                                 
Total params: 1811 (7.07 KB)
Trainable params: 1811 (7.07 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [67]:

# Compile the model (THIRD ATTEMPT)
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [68]:
# Train the model (THIRD ATTEMPT)
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5906 - accuracy: 0.7115
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5808 - accuracy: 0.7246
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5673 - accuracy: 0.7279
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5715 - accuracy: 0.7297
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5605 - accuracy: 0.7303
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5708 - accuracy: 0.7244
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5546 - accuracy: 0.7308
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5555 - accuracy: 0.7307
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5526 - accuracy: 0.7305
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5575 - accura

In [69]:
# Evaluate the model using the test data (THIRD ATTEMPT)
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5694 - accuracy: 0.7287 - 506ms/epoch - 2ms/step
Loss: 0.569409191608429, Accuracy: 0.7287463545799255


In [70]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")